In [94]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import sklearn

In [95]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_206e572456504b2294b20e4c53b9e3b4 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='nh3-_dRkH8hP1r68Cs0-_MnX2q0Otrfr881Gb7Z9EQ2c',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_206e572456504b2294b20e4c53b9e3b4.get_object(Bucket='lifeexpectancy-donotdelete-pr-m5byqj0qrb6pfd',Key='Life Expectancy Data.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
dataset= pd.read_csv(body)
dataset.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

dataset = dataset.drop(['Year','Country'],axis=1)

dataset.head()
status = pd.get_dummies(dataset.Status)
dataset = pd.concat([dataset, status], axis = 1)
dataset = dataset.drop(['Status'],axis=1)
dataset.head()
dataset.rename(columns = {'Developing' : '0', 'Developed' : '1'})
dataset.columns
y = dataset['Life expectancy ']
X = dataset.drop(['Life expectancy '],axis=1)

X.isnull().sum()
y.isnull().sum()
X.fillna(value=X.mean(),inplace=True)
y.fillna(value=y.mean(),inplace=True)
y.isnull().sum()
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0) 

from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)
X_train
dataset.columns

Index(['Life expectancy ', 'Adult Mortality', 'infant deaths', 'Alcohol',
       'percentage expenditure', 'Hepatitis B', 'Measles ', ' BMI ',
       'under-five deaths ', 'Polio', 'Total expenditure', 'Diphtheria ',
       ' HIV/AIDS', 'GDP', 'Population', ' thinness  1-19 years',
       ' thinness 5-9 years', 'Income composition of resources', 'Schooling',
       'Developed', 'Developing'],
      dtype='object')

In [79]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [80]:
# Predicting a new result
y_pred = regressor.predict(min_max_scaler.fit_transform(X_test))
np.set_printoptions(precision=2)
y_pred = np.array(y_pred)
y_test = np.array(y_test)

print(np.concatenate((y_pred.reshape(len(y_test),1), y_test.reshape(len(y_test),1)),1))

[[78.68 77.3 ]
 [66.58 66.6 ]
 [51.97 52.6 ]
 ...
 [72.85 72.6 ]
 [74.33 73.  ]
 [82.02 83.  ]]


In [81]:
accuracy_score =  regressor.score(X_train,y_train)
print(accuracy_score)

0.9948537850177617


In [82]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
print("Mean absolute error: %.2f"
      % mean_absolute_error(y_test, y_pred))
print('R_square score: %.2f' % r2_score(y_test, y_pred))

Mean squared error: 4.69
Mean absolute error: 1.42
R_square score: 0.95


In [83]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(regressor,X_train,y_train,cv=10)
accuracies.mean()

0.959963167350355

In [84]:
!pip install watson-machine-learning-client

In [85]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

wml_credentials={
    "apikey": "DvUOrp6_AnmVFHcyq7A-_sWD248A2fkcJVQS0Zq488cs",
    "iam_apikey_description": "Auto-generated for key 4b9b5798-f082-49c8-836f-635398895aab",
    "iam_apikey_name": "Service credentials-1",
    "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
    "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/f90bc23c8e01478cbc31f846bc9b96b6::serviceid:ServiceId-ddd8228a-ad55-45e5-abea-bb4abe0e3e75",
    "instance_id": "59db06a3-b761-48e5-9c62-f5268cb529a6",
    "url": "https://us-south.ml.cloud.ibm.com"
}

client = WatsonMachineLearningAPIClient( wml_credentials)



In [86]:
model_props = {
    client.repository.ModelMetaNames.AUTHOR_NAME : "Yashwanth",
    client.repository.ModelMetaNames.AUTHOR_EMAIL : "si05202000098@smartinternz.com",
    client.repository.ModelMetaNames.NAME : "life-expectancy"
}

In [90]:
model_artifact =client.repository.store_model(regressor, meta_props=model_props)
model_artifact

{'metadata': {'guid': '153bc6af-0eca-4f57-a679-bd549967d86a',
  'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/59db06a3-b761-48e5-9c62-f5268cb529a6/published_models/153bc6af-0eca-4f57-a679-bd549967d86a',
  'created_at': '2020-06-06T05:59:33.235Z',
  'modified_at': '2020-06-06T05:59:33.303Z'},
 'entity': {'runtime_environment': 'python-3.6',
  'learning_configuration_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/59db06a3-b761-48e5-9c62-f5268cb529a6/published_models/153bc6af-0eca-4f57-a679-bd549967d86a/learning_configuration',
  'author': {'name': 'Yashwanth'},
  'name': 'life-expectancy',
  'learning_iterations_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/59db06a3-b761-48e5-9c62-f5268cb529a6/published_models/153bc6af-0eca-4f57-a679-bd549967d86a/learning_iterations',
  'feedback_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/59db06a3-b761-48e5-9c62-f5268cb529a6/published_models/153bc6af-0eca-4f57-a679-bd549967d86a/feedback',
  'latest_versi

In [91]:
published_model_uid=client.repository.get_model_uid(model_artifact) 
published_model_uid

'153bc6af-0eca-4f57-a679-bd549967d86a'

In [92]:
deployment =client.deployments.create(published_model_uid, name="Life_Expectancy_RandomForest") 
scoring_endpoint =client.deployments.get_scoring_url(deployment)
scoring_endpoint





#######################################################################################

Synchronous deployment creation for uid: '153bc6af-0eca-4f57-a679-bd549967d86a' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='d93bc168-d6f9-4693-af50-22ae5584516d'
------------------------------------------------------------------------------------------------




'https://us-south.ml.cloud.ibm.com/v3/wml_instances/59db06a3-b761-48e5-9c62-f5268cb529a6/deployments/d93bc168-d6f9-4693-af50-22ae5584516d/online'